### Check how the FB system responded during https://jira.lsstcorp.org/browse/SUMMIT-5078

With M1M3 sitting at zenith, we let it use mount telemetry elevatino angle to determine its elevation forces. 
The created imbalance in the system, which we relied on the FB system to cancel out.
The system faulted with elevation angle at about 79.5 deg. Actuator 319 near-neighbor check was triggered.

In [ ]:
summit = 1 #use this for summit testing
#summit = 0 #use this for NCSA

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd

from lsst.ts.idl.enums import MTM1M3
from M1M3tools import *
from lsst_efd_client import EfdClient

plt.jet();

In [ ]:
if summit:
    client = EfdClient('summit_efd')
else:
    client = EfdClient('ncsa_teststand_efd')
csc_index = 1

In [ ]:
end = Time('2021-04-01T16:06:00')
start = end - timedelta(minutes=5)
df = await client.select_time_series('lsst.sal.MTM1M3.forceActuatorData', '*', 
                                     (start-timedelta(seconds=37)).tai, (end-timedelta(seconds=37)).tai, csc_index)
dfm = await client.select_time_series('lsst.sal.MTMount.elevation', '*', 
                                     (start-timedelta(seconds=37)).tai, (end-timedelta(seconds=37)).tai, csc_index)

### Find out the exact elevation angle where we faulted

In [ ]:
#find where we started faulting. 170000 is mirror weight in Newton
aa = np.array(df.forceMagnitude)
idx = np.argmax(aa<170000)
tt = df.private_sndStamp[idx] #mount telemetry was published at 1Hz
idxm = np.logical_and(dfm.private_sndStamp >tt-1, dfm.private_sndStamp <tt+1)

In [ ]:
plt.plot(np.arange(20), df.forceMagnitude[idx-10:idx+10],'-o')
plt.grid();

In [ ]:
#what was the mount elevation angle when we faulted
dfm.actualPosition[idxm]

### Check the forces at the time we faulted

In [ ]:
idIdx = []
#https://github.com/lsst-ts/ts_m1m3support/blob/develop/SettingFiles/Tables/ForceActuatorNearNeighborZTable.csv
myID = [319, 312, 318, 325, 311] 
for i in myID:
    idIdx.append(np.argmax(actID==i))

In [ ]:
testIdx = idx
for i, j in zip(idIdx,myID):
    print(j, fat[i, FATABLE_TYPE], end = ' ')
    print(df['zForce%d'%i][testIdx])
    if j == 319:
        centerF = df['zForce%d'%i][testIdx]
        outsideF = []
    else:
        outsideF.append(df['zForce%d'%i][testIdx])
print('near neighbor check: ', centerF - np.mean(outsideF))

### Check how the forces changed during the entire process

In [ ]:
end = Time('2021-04-01T16:06:00')
start = end - timedelta(minutes=40)
dflist = []
for i in idIdx:
    print(i)
    dflist.append(await client.select_time_series('lsst.sal.MTM1M3.forceActuatorData', 'zForce%d'%i, 
                                     (start-timedelta(seconds=37)).tai, (end-timedelta(seconds=37)).tai, csc_index))

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,4))
for i,j in enumerate(myID):
    plt.plot(dflist[i], label='%d'%j)
plt.plot(dflist[0] - np.mean(dflist[1:]), label='near-neighbor diff')
plt.legend()

In [ ]:
dfm = await client.select_time_series('lsst.sal.MTMount.elevation', '*', 
                                     (start-timedelta(seconds=37)).tai, (end-timedelta(seconds=37)).tai, csc_index)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,4))
plt.plot(dfm.actualPosition)
plt.grid();

319, 311, 318 forces were decreasing. While 312, 325 forces were increasing. This feels a bit strange. They are all at negative x and negative y. The balance force distributions are all linear. Why would that affect these 5 actuators differetly.

![title](fault319.png)

In [ ]:
dflist = []
for i in idIdx:
    print(i)
    dflist.append(await client.select_time_series('lsst.sal.MTM1M3.logevent_appliedElevationForces', 'zForces%d'%i, 
                                     (start-timedelta(seconds=37)).tai, (end-timedelta(seconds=37)).tai, csc_index))
fig, ax = plt.subplots(1,1, figsize=(15,4))
for i,j in enumerate(myID):
    plt.plot(dflist[i], label='%d'%j)
plt.legend()

In [ ]:
dflist = []
for i in idIdx:
    print(i)
    dflist.append(await client.select_time_series('lsst.sal.MTM1M3.logevent_appliedBalanceForces', 'zForces%d'%i, 
                                     (start-timedelta(seconds=37)).tai, (end-timedelta(seconds=37)).tai, csc_index))
fig, ax = plt.subplots(1,1, figsize=(15,4))
for i,j in enumerate(myID):
    plt.plot(dflist[i], label='%d'%j)
plt.legend()